<a href="https://colab.research.google.com/github/Vishnu0920/Recommender_System_Models/blob/main/AttentionalFMModified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [ ]:
# Load the dataset
df = pd.read_csv('grade_data.csv')

# Encode student_id and course_id using LabelEncoder
le_student = LabelEncoder()
le_course = LabelEncoder()

df['student_id'] = le_student.fit_transform(df['student_id'])
df['course_id'] = le_course.fit_transform(df['course_id'])

# Map course grades to the range [0, 1] for regression
df['course_grade'] = df['course_grade'] / 10.0

# Train-test split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


In [ ]:
# Convert the data to PyTorch tensors
train_user = torch.LongTensor(train_df['student_id'].values)
train_course = torch.LongTensor(train_df['course_id'].values)
train_grade = torch.FloatTensor(train_df['course_grade'].values)

test_user = torch.LongTensor(test_df['student_id'].values)
test_course = torch.LongTensor(test_df['course_id'].values)
test_grade = torch.FloatTensor(test_df['course_grade'].values)


In [ ]:
class AttentionalFactorizationMachine(nn.Module):
    def __init__(self, num_courses, embedding_dim):
        super(AttentionalFactorizationMachine, self).__init__()

        # Adjust the dimensions of embeddings and linear layer
        self.course_embedding = nn.Embedding(num_courses, embedding_dim)
        self.linear = nn.Linear(embedding_dim, 1)
        self.attention = nn.Linear(embedding_dim, 1)

    def forward(self, course):
        course_emb = self.course_embedding(course)

        # Attention mechanism
        attention_weights = torch.sigmoid(self.attention(course_emb))
        attention = attention_weights * course_emb

        # Concatenate course embeddings with attention
        course_attention = course_emb + attention

        output = self.linear(course_attention)
        return output.squeeze()


In [ ]:
# Hyperparameters
num_users = len(le_student.classes_)
num_courses = len(le_course.classes_)
embedding_dim = 10
num_epochs = 10
batch_size = 64
learning_rate = 0.001

# Create DataLoader for training
train_dataset = TensorDataset(train_user, train_course, train_grade)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Instantiate the model, define loss function and optimizer
model = AttentionalFactorizationMachine(num_courses, embedding_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
# Set the print interval
print_interval = 10  # Adjust the interval as needed

# Cell 12: Training loop with performance monitoring
for epoch in range(num_epochs):
    total_loss = 0.0
    for batch_index, batch in enumerate(train_dataloader):
        _, course, grade = batch  # Remove user variable

        optimizer.zero_grad()
        output = model(course).squeeze()
        loss = criterion(output, grade)
        loss.backward()
        optimizer.step()

        if batch_index % print_interval == 0:
            print(f'Epoch {epoch}, Batch {batch_index}, Loss: {loss.item()}')


Epoch 0, Batch 0, Loss: 2.164921283721924
Epoch 0, Batch 10, Loss: 1.6263986825942993
Epoch 0, Batch 20, Loss: 2.104883909225464
Epoch 0, Batch 30, Loss: 1.5321447849273682
Epoch 0, Batch 40, Loss: 1.8873634338378906
Epoch 0, Batch 50, Loss: 1.516208529472351
Epoch 0, Batch 60, Loss: 1.7176568508148193
Epoch 0, Batch 70, Loss: 1.2444063425064087
Epoch 0, Batch 80, Loss: 1.1137255430221558
Epoch 0, Batch 90, Loss: 0.9756931662559509
Epoch 0, Batch 100, Loss: 1.1234780550003052
Epoch 0, Batch 110, Loss: 1.0259339809417725
Epoch 0, Batch 120, Loss: 1.0519884824752808
Epoch 0, Batch 130, Loss: 0.9035086035728455
Epoch 0, Batch 140, Loss: 0.9574010372161865
Epoch 0, Batch 150, Loss: 0.8913798928260803
Epoch 0, Batch 160, Loss: 0.8093476295471191
Epoch 0, Batch 170, Loss: 0.8353613615036011
Epoch 0, Batch 180, Loss: 0.7585614323616028
Epoch 0, Batch 190, Loss: 0.757659912109375
Epoch 0, Batch 200, Loss: 0.6633107662200928
Epoch 0, Batch 210, Loss: 0.6984236836433411
Epoch 0, Batch 220, Loss:

In [ ]:
# Evaluation on the test dataset
model.eval()
# Define test dataset
test_dataset = TensorDataset(test_user, test_course, test_grade)
# Define test dataloader
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

total_test_loss = 0.0
num_test_samples = len(test_dataset)

with torch.no_grad():
    for batch in test_dataloader:
        user,course, grade = batch
        output = model(course).squeeze()
        loss = criterion(output, grade)
        total_test_loss += loss.item()

test_mse = total_test_loss / num_test_samples
print(f'Mean Squared Error (MSE) on the test dataset: {test_mse}')


Mean Squared Error (MSE) on the test dataset: 0.000335361745078151


In [ ]:
# Recommender function modified to exclude user_id
def recommend_courses(model, input_courses, top_k):
    # Convert input_courses to PyTorch tensor
    input_course_ids = [le_course.transform([course])[0] for course in input_courses.keys()]
    input_course_grades = list(input_courses.values())

    # Make predictions for all courses
    all_course_ids = torch.arange(len(le_course.classes_))
    predictions = model(all_course_ids).squeeze()

    # Exclude input courses from recommendations
    for course_id in input_course_ids:
        predictions[course_id] = float('-inf')

    # Get the indices of the top-k predictions
    num_recommendations = min(top_k, len(predictions))
    top_indices = torch.topk(predictions, num_recommendations).indices

    # Map the top indices back to the course IDs
    top_course_ids = le_course.inverse_transform(top_indices.numpy())

    # Exclude input courses from recommendations (additional check)
    top_course_ids = [course_id for course_id in top_course_ids if course_id not in input_courses]

    return top_course_ids


In [ ]:
# Example: Input student's grade for previous courses
input_courses = {
    'CHEMISTRY LABORATORY': 8,
    'GENERAL CHEMISTRY': 10,
    'ELECTRICAL SCIENCES':10,
    'ADDITIVE MANUFACTURING': 10,
    'PRACTICE SCHOOL I':10,
    'PHYSICS LABORATORY':10
}

# Set top_k to the desired number
top_k = 5

# Call the function with the updated course grades
recommended_courses = recommend_courses(model, input_courses, top_k)

print(f"Top {top_k} recommended courses based on previous grades: {recommended_courses}")

Top 5 recommended courses based on previous grades: ['PRACTICE SCHOOL II', 'ELEC MAGNET & OPTICS LAB', 'THESIS', 'ELEC & ELECTRONIC CIRCUITS LAB', 'LABORATORY PROJECT']
